In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("concat_data.csv",low_memory =False)

In [3]:
df = df.drop(['Unnamed: 0'], axis = 1)

In [5]:
df.head()

,기준일자,기준시간대값,시군구명,행정동명,카드사명,업종명,출생년도,성별코드,거래건수,총거래금액,age,age_range,요일
0,2022-06-01,0,중구,대청동,코나,편 의 점,1973,M,1,7350,50,50,Wednesday
1,2022-06-01,0,중구,중앙동,부산,편 의 점,1981,F,1,3700,42,40,Wednesday
2,2022-06-01,0,중구,중앙동,부산,편 의 점,1992,M,1,7500,31,30,Wednesday
3,2022-06-01,0,중구,중앙동,부산,편 의 점,1995,M,1,2700,28,20,Wednesday
4,2022-06-01,0,중구,중앙동,코나,편 의 점,1976,M,1,13900,47,40,Wednesday


In [6]:
# 거래 합산 후 건별 평균 금액
df['건별거래금액'] = df['총거래금액']/df['거래건수']
len(df)

27135019

In [7]:
df.drop(df[(df['건별거래금액'] < 10)].index, inplace=True)
len(df)

27134908

In [9]:
df.to_csv('concat_data_new.csv',index=False)

In [10]:
df = pd.read_csv("concat_data_new.csv",low_memory =False)

In [12]:
population = pd.read_csv("busan_population.csv", encoding = 'cp949', low_memory=False)
population

,region,june,july,august
0,중구,42692,42594,42455
1,서구,106397,106818,107522
2,동구,90186,89944,89800
3,영도구,111307,111271,111100
4,부산진구,353674,354120,354367
5,동래구,275668,275956,275830
6,남구,265158,264557,264112
7,북구,282625,282381,282230
8,해운대구,396120,395601,394626
9,사하구,309199,308777,308286


In [14]:
import folium

In [229]:
store = pd.read_csv("부산광역시_지역화폐(동백전) 가맹점 현황_20221222.csv",encoding = 'cp949', low_memory =False)
store.head()

,순번,가맹점명,도로명,지번,위도,경도,데이터기준일자
0,1,고봉민김밥인,부산 영도구 와치로 250 (동삼동) 1층,NaN,35.075315,129.068770,2022-12-22
1,2,미사랑스킨앤바디,"부산 사하구 승학로224번길 10 ,1층(괴정동)",NaN,35.101239,128.990313,2022-12-22
2,3,피자스쿨 부산재송점,부산 해운대구 재반로 182 비둘기맨션상가 B동 가호 (재송동),NaN,35.193467,129.130557,2022-12-22
3,4,미스터피자아시아드2호점,"부산 연제구 월드컵대로 232 2층 (거제동,거벌빌등)",NaN,35.189890,129.071940,2022-12-22
4,5,성원칼국수,부산 부산진구 냉정로 220-1 (개금동),NaN,35.150897,129.023928,2022-12-22


In [230]:
store = store.drop(['순번'], axis = 1)

In [231]:
# 도로명 대신 지번 채워진 결측치 합쳐서 주소 컬럼 생성
store['주소'] = store['도로명']
store['주소'] = store['도로명'].fillna(store['지번'])

In [232]:
# 결측치 채워졌나 확인
store['주소'].isnull().sum()

0

In [233]:
# 사업자등록번호 때문에 부산 주소만
store = store[store['주소'].str.startswith('부산')]

In [219]:
# 부산광역시 수정 됨.
store['주소'].str.replace('부산광역시','부산')

0                                  부산 영도구 와치로 250 (동삼동) 1층
1                              부산 사하구 승학로224번길 10 ,1층(괴정동)
2                      부산 해운대구 재반로 182 비둘기맨션상가 B동 가호 (재송동)
3                           부산 연제구 월드컵대로 232 2층 (거제동,거벌빌등)
4                                  부산 부산진구 냉정로 220-1 (개금동)
                                ...                       
126936    부산 강서구  명지오션시티11로 22, 102동 203호 (명지동,명지두산위브포세이돈)
126937          부산 사하구 서포로30번길 12 207동 301호 (구평동,e편한세상아파트)
126938                           부산 중구  중앙대로 137-1 (대창동2가)
126939                                  부산 사상구 덕포동 404-7,8
126940                              부산 연제구 월드컵대로 399 (거제동)
Name: 주소, Length: 126941, dtype: object

In [228]:
store.loc[399]

'부산 동래구 명서로 182 상가 9호동 3층 301호'

In [158]:
# 부산에 있는 매장만
store1 = store[store['주소'].str.startswith('부산 ')]
store1

,가맹점명,도로명,지번,위도,경도,데이터기준일자,주소
0,고봉민김밥인,부산 영도구 와치로 250 (동삼동) 1층,NaN,35.075315,129.068770,2022-12-22,부산 영도구 와치로 250 (동삼동) 1층
1,미사랑스킨앤바디,"부산 사하구 승학로224번길 10 ,1층(괴정동)",NaN,35.101239,128.990313,2022-12-22,"부산 사하구 승학로224번길 10 ,1층(괴정동)"
2,피자스쿨 부산재송점,부산 해운대구 재반로 182 비둘기맨션상가 B동 가호 (재송동),NaN,35.193467,129.130557,2022-12-22,부산 해운대구 재반로 182 비둘기맨션상가 B동 가호 (재송동)
3,미스터피자아시아드2호점,"부산 연제구 월드컵대로 232 2층 (거제동,거벌빌등)",NaN,35.189890,129.071940,2022-12-22,"부산 연제구 월드컵대로 232 2층 (거제동,거벌빌등)"
4,성원칼국수,부산 부산진구 냉정로 220-1 (개금동),NaN,35.150897,129.023928,2022-12-22,부산 부산진구 냉정로 220-1 (개금동)
...,...,...,...,...,...,...,...
126936,행복한 정쌤 공부방,"부산 강서구 명지오션시티11로 22, 102동 203호 (명지동,명지두산위브포세이돈)",NaN,35.083390,128.896842,2022-12-22,"부산 강서구 명지오션시티11로 22, 102동 203호 (명지동,명지두산위브포세이돈)"
126937,행복홈케어,"부산 사하구 서포로30번길 12 207동 301호 (구평동,e편한세상아파트)",NaN,35.081755,128.985395,2022-12-22,"부산 사하구 서포로30번길 12 207동 301호 (구평동,e편한세상아파트)"
126938,현대표구사,부산 중구 중앙대로 137-1 (대창동2가),NaN,35.109934,129.037028,2022-12-22,부산 중구 중앙대로 137-1 (대창동2가)
126939,현혜반점,NaN,"부산 사상구 덕포동 404-7,8",NaN,NaN,2022-12-22,"부산 사상구 덕포동 404-7,8"


In [159]:
# 인덱스 리셋
store1 = store1.reset_index(drop=True)
store1

,가맹점명,도로명,지번,위도,경도,데이터기준일자,주소
0,고봉민김밥인,부산 영도구 와치로 250 (동삼동) 1층,NaN,35.075315,129.068770,2022-12-22,부산 영도구 와치로 250 (동삼동) 1층
1,미사랑스킨앤바디,"부산 사하구 승학로224번길 10 ,1층(괴정동)",NaN,35.101239,128.990313,2022-12-22,"부산 사하구 승학로224번길 10 ,1층(괴정동)"
2,피자스쿨 부산재송점,부산 해운대구 재반로 182 비둘기맨션상가 B동 가호 (재송동),NaN,35.193467,129.130557,2022-12-22,부산 해운대구 재반로 182 비둘기맨션상가 B동 가호 (재송동)
3,미스터피자아시아드2호점,"부산 연제구 월드컵대로 232 2층 (거제동,거벌빌등)",NaN,35.189890,129.071940,2022-12-22,"부산 연제구 월드컵대로 232 2층 (거제동,거벌빌등)"
4,성원칼국수,부산 부산진구 냉정로 220-1 (개금동),NaN,35.150897,129.023928,2022-12-22,부산 부산진구 냉정로 220-1 (개금동)
...,...,...,...,...,...,...,...
124369,행복한 정쌤 공부방,"부산 강서구 명지오션시티11로 22, 102동 203호 (명지동,명지두산위브포세이돈)",NaN,35.083390,128.896842,2022-12-22,"부산 강서구 명지오션시티11로 22, 102동 203호 (명지동,명지두산위브포세이돈)"
124370,행복홈케어,"부산 사하구 서포로30번길 12 207동 301호 (구평동,e편한세상아파트)",NaN,35.081755,128.985395,2022-12-22,"부산 사하구 서포로30번길 12 207동 301호 (구평동,e편한세상아파트)"
124371,현대표구사,부산 중구 중앙대로 137-1 (대창동2가),NaN,35.109934,129.037028,2022-12-22,부산 중구 중앙대로 137-1 (대창동2가)
124372,현혜반점,NaN,"부산 사상구 덕포동 404-7,8",NaN,NaN,2022-12-22,"부산 사상구 덕포동 404-7,8"


In [172]:
# 시군구명 추출
for i in range(0,len(store1)):
    store1.loc[i, '시군구명'] = store1.loc[i, '주소'][3:store1.loc[i,'주소'].find(' ', 3)]

In [180]:
# 빈 값 확인 (결측치는 아님.)
store1['시군구명'].sort_values()

74640         
12728         
36585      강서구
116491     강서구
65641      강서구
          ... 
54454     해운대구
54457     해운대구
54461     해운대구
54444     해운대구
110928    해운대구
Name: 시군구명, Length: 124374, dtype: object

In [187]:
# 오류 찾음
store1.loc[74640, :]

가맹점명                           보광훼미리마트경성대센츄리점
도로명        부산   남구   대연동 55-1 21 센츄리오피스텔 201 
지번                                        NaN
위도                                  35.136983
경도                                  129.10062
데이터기준일자                            2022-12-22
주소         부산   남구   대연동 55-1 21 센츄리오피스텔 201 
시군구명                                       남구
Name: 74640, dtype: object

In [186]:
store1.loc[74640, '시군구명'] = '남구'

In [189]:
store1.loc[12728, '시군구명'] = '사하구'

In [215]:
store1.iloc[:, [0,3,4,5,6,7]]

,가맹점명,위도,경도,데이터기준일자,주소,시군구명
0,고봉민김밥인,35.075315,129.068770,2022-12-22,부산 영도구 와치로 250 (동삼동) 1층,영도구
1,미사랑스킨앤바디,35.101239,128.990313,2022-12-22,"부산 사하구 승학로224번길 10 ,1층(괴정동)",사하구
2,피자스쿨 부산재송점,35.193467,129.130557,2022-12-22,부산 해운대구 재반로 182 비둘기맨션상가 B동 가호 (재송동),해운대구
3,미스터피자아시아드2호점,35.189890,129.071940,2022-12-22,"부산 연제구 월드컵대로 232 2층 (거제동,거벌빌등)",연제구
4,성원칼국수,35.150897,129.023928,2022-12-22,부산 부산진구 냉정로 220-1 (개금동),부산진구
...,...,...,...,...,...,...
124369,행복한 정쌤 공부방,35.083390,128.896842,2022-12-22,"부산 강서구 명지오션시티11로 22, 102동 203호 (명지동,명지두산위브포세이돈)",강서구
124370,행복홈케어,35.081755,128.985395,2022-12-22,"부산 사하구 서포로30번길 12 207동 301호 (구평동,e편한세상아파트)",사하구
124371,현대표구사,35.109934,129.037028,2022-12-22,부산 중구 중앙대로 137-1 (대창동2가),중구
124372,현혜반점,NaN,NaN,2022-12-22,"부산 사상구 덕포동 404-7,8",사상구


In [190]:
# 정제 완료
store_count = store1['시군구명'].value_counts().to_frame()
store_count

,시군구명
부산진구,15716
해운대구,12804
동래구,10302
사하구,9384
금정구,8567
남구,8117
북구,7880
사상구,7869
수영구,7729
연제구,7398


In [191]:
import folium as g
import json
import webbrowser
import pandas as pd

g_map = g.Map(location=[35.075315,129.068770], 
            tiles='cartodbpositron', 
            # tiles = 'Stamen Terrain',
            zoom_start=10)
geo_data = json.load(open('skorea_municipalities_geo_simple.json', encoding='utf-8'))

In [198]:
b = pd.read_csv("busan.csv",encoding = 'cp949',index_col=0)
b

,price,population,area,density
gu,,,,
중구,15515,44852,2.83,15849
서구,28665,112621,13.98,8056
동구,20729,89144,9.74,9152
영도구,17024,121934,14.20,8587
부산진구,28781,365337,29.67,12313
동래구,35607,267735,16.63,16100
남구,29597,286093,26.81,10671
북구,23090,299547,39.37,7609
해운대구,34649,414611,51.47,8055


In [59]:
g.Choropleth(geo_data=geo_data,
                data = df,
                columns=['sigun', 'avg_income'],
                key_on='feature.properties.name',
                fill_color='YlGn',
                fill_opacity=0.8
            ).add_to(g_map)
            

In [216]:
geo_path = "./busan_gu.json"
geo_str = json.load(open(geo_path, encoding='utf-8'))
map = folium.Map(location = [35.1856505,129.1], zoom_start=10, tiles="cartodb positron")
map.choropleth(fill_opacity=0.5,geo_data = geo_str, 
               data = store_count['시군구명'],
              columns = [store_count.index, store_count["시군구명"]],
              fill_color = "YlOrRd",
              key_on = "feature.id")

map

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/folium/folium.py:465: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(


In [206]:
c = choropleth(fill_opacity=0.5,geo_data = geo_str, 
               data = store_count['시군구명'],
              columns = [store_count.index, store_count["시군구명"]],
              fill_color = "RdYlGn",
              key_on = "feature.id")

NameError: name 'choropleth' is not defined

In [46]:
g_map.save('map.html')
webbrowser.open_new_tab('map.html')

True